In [1]:
%matplotlib inline


Introduction to TOPI
====================
**Author**: `Ehsan M. Kermani <https://github.com/ehsanmok>`_

This is an introductory tutorial to TVM Operator Inventory (TOPI).
TOPI provides numpy-style generic operations and schedules with higher abstractions than TVM.
In this tutorial, we will see how TOPI can save us from writing boilerplates code in TVM.



In [2]:
from __future__ import absolute_import, print_function

import tvm
from tvm import te
import topi
import numpy as np

Basic example
-------------
Let's revisit the sum of rows operation (equivalent to :code:`B = numpy.sum(A, axis=1)`') \
To compute the sum of rows of a two dimensional TVM tensor A, we should
specify the symbolic operation as well as schedule as follows




In [4]:
n = te.var("n")
m = te.var("m")
A = te.placeholder((n, m), name='A')
k = te.reduce_axis((0, m), "k")
B = te.compute((n,), lambda i: te.sum(A[i, k], axis=k), name="B")
s = te.create_schedule(B.op)

and to examine the IR code in human readable format, we can do




In [5]:
print(tvm.lower(s, [A], simple_mode=True))

// attr [B] storage_scope = "global"
allocate B[float32 * n]
produce B {
  for (i, 0, n) {
    B[i] = 0f
    for (k, 0, m) {
      B[i] = (B[i] + A[((i*stride) + (k*stride))])
    }
  }
}



However, for such a common operation we had to define the reduce axis ourselves as well as explicit computation with
:code:`te.compute`. Imagine for more complicated operations how much details we need to provide.
Fortunately, we can replace those two lines with simple :code:`topi.sum` much like :code:`numpy.sum`




In [7]:
C = topi.sum(A, axis=1)
ts = te.create_schedule(C.op)
print(tvm.lower(ts, [A], simple_mode=True))

// attr [A_red] storage_scope = "global"
allocate A_red[float32 * n]
produce A_red {
  for (ax0, 0, n) {
    A_red[ax0] = 0f
    for (k1, 0, m) {
      A_red[ax0] = (A_red[ax0] + A[((ax0*stride) + (k1*stride))])
    }
  }
}



Numpy-style operator overloading
--------------------------------
We can add two tensors using :code:`topi.broadcast_add` that have correct (broadcastable with specific) shapes.
Even shorter, TOPI provides operator overloading for such common operations. For example,




In [8]:
x, y = 100, 10
a = te.placeholder((x, y, y), name="a")
b = te.placeholder((y, y), name="b")
c = a + b  # same as topi.broadcast_add
d = a * b  # same as topi.broadcast_mul

Overloaded with the same syntax, TOPI handles broadcasting a primitive (`int`, `float`) to a tensor :code:`d - 3.14`.



Generic schedules and fusing operations
---------------------------------------
Up to now, we have seen an example of how TOPI can save us from writing explicit computations in lower level API.
But it doesn't stop here. Still we did the scheduling as before. TOPI also provides higher level
scheduling recipes depending on a given context. For example, for CUDA,
we can schedule the following series of operations ending with :code:`topi.sum` using only
:code:`topi.generic.schedule_reduce`




In [9]:
e = topi.elemwise_sum([c, d])
f = e / 2.0
g = topi.sum(f)
with tvm.target.cuda():
    sg = topi.cuda.schedule_reduce(g)
    print(tvm.lower(sg, [a, b], simple_mode=True))

// attr [T_divide_red] storage_scope = "global"
allocate T_divide_red[float32 * 1]
produce T_divide_red {
  // attr [iter_var(threadIdx.x, range(min=0, ext=1024), threadIdx.x)] thread_extent = 1024
  // attr [T_divide_red.rf] storage_scope = "local"
  allocate T_divide_red.rf[float32 * 1]
  // attr [reduce_temp0] storage_scope = "local"
  allocate reduce_temp0[float32 * 1]
  produce T_divide_red.rf {
    T_divide_red.rf[0] = 0f
    for (k0.k1.fused.k2.fused.outer, 0, 10) {
      if (likely((((((k0.k1.fused.k2.fused.outer*1024) + threadIdx.x) < 10000) && (((k0.k1.fused.k2.fused.outer*1024) + threadIdx.x) < 10000)) && (((k0.k1.fused.k2.fused.outer*1024) + threadIdx.x) < 10000)))) {
        T_divide_red.rf[0] = (T_divide_red.rf[0] + (((a[((k0.k1.fused.k2.fused.outer*1024) + threadIdx.x)] + b[floormod(((k0.k1.fused.k2.fused.outer*1024) + threadIdx.x), 100)]) + (a[((k0.k1.fused.k2.fused.outer*1024) + threadIdx.x)]*b[floormod(((k0.k1.fused.k2.fused.outer*1024) + threadIdx.x), 100)]))*0.5f))


As you can see, scheduled stages of computation have been accumulated and we can examine them by




In [10]:
print(sg.stages)

[stage(a, 0x3479de0), stage(b, 0x2d33c90), stage(T_add, 0x3bbff50), stage(T_multiply, 0x3b6c620), stage(T_elemwise_sum, 0x3a23b90), stage(T_divide, 0x3b6b240), stage(T_divide_red.rf, 0x3c44220), stage(T_divide_red, 0x3c41040)]


We can test the correctness by comparing with :code:`numpy` result as follows




In [11]:
func = tvm.build(sg, [a, b, g], 'cuda')
ctx = tvm.gpu(0)
a_np = np.random.uniform(size=(x, y, y)).astype(a.dtype)
b_np = np.random.uniform(size=(y, y)).astype(b.dtype)
g_np = np.sum(np.add(a_np + b_np, a_np * b_np) / 2.0)
a_nd = tvm.nd.array(a_np, ctx)
b_nd = tvm.nd.array(b_np, ctx)
g_nd = tvm.nd.array(np.zeros(g_np.shape, dtype=g_np.dtype), ctx)
func(a_nd, b_nd, g_nd)
tvm.testing.assert_allclose(g_nd.asnumpy(), g_np, rtol=1e-5)

TOPI also provides common neural nets operations such as _softmax_ with optimized schedule




In [12]:
tarray = te.placeholder((512, 512), name="tarray")
softmax_topi = topi.nn.softmax(tarray)
with tvm.target.create("cuda"):
    sst = topi.cuda.schedule_softmax(softmax_topi)
    print(tvm.lower(sst, [tarray], simple_mode=True))

// attr [T_softmax_maxelem] storage_scope = "global"
allocate T_softmax_maxelem[float32 * 512]
// attr [T_softmax_exp] storage_scope = "global"
allocate T_softmax_exp[float32 * 262144]
produce T_softmax_maxelem {
  // attr [iter_var(blockIdx.x, , blockIdx.x)] thread_extent = 512
  T_softmax_maxelem[blockIdx.x] = -3.40282e+38f
  for (k, 0, 512) {
    T_softmax_maxelem[blockIdx.x] = max(T_softmax_maxelem[blockIdx.x], tarray[((blockIdx.x*512) + k)])
  }
}
produce T_softmax_exp {
  // attr [iter_var(blockIdx.x, , blockIdx.x)] thread_extent = 512
  for (i1, 0, 512) {
    T_softmax_exp[((blockIdx.x*512) + i1)] = exp((tarray[((blockIdx.x*512) + i1)] - T_softmax_maxelem[blockIdx.x]))
  }
}
produce T_softmax_expsum {
  // attr [iter_var(blockIdx.x, , blockIdx.x)] thread_extent = 512
  // attr [T_softmax_expsum.rf] storage_scope = "local"
  allocate T_softmax_expsum.rf[float32 * 1]
  // attr [reduce_temp0] storage_scope = "local"
  allocate reduce_temp0[float32 * 1]
  // attr [iter_var(threadIdx

Fusing convolutions
-------------------
We can fuse :code:`topi.nn.conv2d` and :code:`topi.nn.relu` together.

<div class="alert alert-info"><h4>Note</h4><p>TOPI functions are all generic functions. They have different implementations
   for different backends to optimize for performance.
   For each backend, it is necessary to call them under a target scope for both
   compute declaration and schedule. TVM will choose the right function to call with
   the target information.</p></div>



In [13]:
data = te.placeholder((1, 3, 224, 224))
kernel = te.placeholder((10, 3, 5, 5))

with tvm.target.create("cuda"):
    conv = topi.cuda.conv2d_nchw(data, kernel, 1, 2, 1)
    out = topi.nn.relu(conv)
    sconv = topi.cuda.schedule_conv2d_nchw([out])
    print(tvm.lower(sconv, [data, kernel], simple_mode=True))

 (compute[12] + (pad_temp.shared[(threadIdx.x + 96)]*placeholder.shared[0]))
        compute[1] = (compute[1] + (pad_temp.shared[threadIdx.x]*placeholder.shared[1]))
        compute[3] = (compute[3] + (pad_temp.shared[(threadIdx.x + 16)]*placeholder.shared[1]))
        compute[5] = (compute[5] + (pad_temp.shared[(threadIdx.x + 32)]*placeholder.shared[1]))
        compute[7] = (compute[7] + (pad_temp.shared[(threadIdx.x + 48)]*placeholder.shared[1]))
        compute[9] = (compute[9] + (pad_temp.shared[(threadIdx.x + 64)]*placeholder.shared[1]))
        compute[11] = (compute[11] + (pad_temp.shared[(threadIdx.x + 80)]*placeholder.shared[1]))
        compute[13] = (compute[13] + (pad_temp.shared[(threadIdx.x + 96)]*placeholder.shared[1]))
        produce pad_temp.shared {
          // attr [iter_var(threadIdx.z, , threadIdx.z)] thread_extent = 1
          // attr [iter_var(threadIdx.y, , threadIdx.y)] thread_extent = 1
          // attr [iter_var(threadIdx.x, , threadIdx.x)] thread_extent

Summary
-------
In this tutorial, we have seen

- How to use TOPI API for common operations with numpy-style operators.
- How TOPI facilitates generic schedules and operator fusion for a context, to generate optimized kernel codes.

